In [1]:
import sys
sys.path.insert(0,'data')
import requests

import utils_db
import utils
import dialogs
import utils_toloka
import time
import vk_api
import config

import logging
logger = logging.getLogger('toloka_loader')
handler = logging.FileHandler('logs/toloka_loader.log',encoding = "UTF-8")
formatter = logging.Formatter('------------\n %(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter); logger.addHandler(handler)

In [3]:
new_images = utils_db.select_new_img_to_toloka()

In [ ]:
def add_new_img_to_toloka(new_images):
    result_post_request = requests.post(url = utils_toloka.url_create_tasks,headers = utils_toloka.headers,
                                        json = utils_toloka.create_json_for_tasks(new_images))
    for task in result_post_request.json()['items'].values():
        image_id = int(task['input_values']['image'][:-4].split('/')[-1])
        utils_db.add_toloka_task_id_in_db(image_id,task['id'])

In [ ]:
while True:
    try:
        st = time.time()
        new_images = utils_db.select_new_img_to_toloka()
        if len(new_images) > 0:
            add_new_img_to_toloka(new_images)
        reviewing_images = utils_db.select_reviewing_images()
        if len(reviewing_images) > 0:
            check_reviewing_images(reviewing_images)
        time.sleep(max(2 - (time.time()-st),0))
    except KeyboardInterrupt:
        break
    except Exception as e:
        logger.exception("error toloka_loader ")
        utils.send_me_error("error toloka_loader ", e)